In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer= namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity
    
class Order:
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
            return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    def __repr__(self):
        fmt = '<Order total: {:2f} due: {:2f}>'
        return fmt.format(self.total(), self.due())
    
class Promotion(ABC):
    
    @abstractmethod
    def discount(self, order):
        """返回折扣金额（正值）"""

class FidelityPromo(Promotion):
    """为积分为1000或以上的顾客提供5%折扣"""
    
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >=1000 else 0

class BulkItemPromo(Promotion):
    """单个商品为20个或以上时提供10%折扣"""
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >=20:
                discount += item.total() *.1
        return discount

class LargeOrderPromo(Promotion):
    """订单中的不同商品达到10个或以上时提供7%折扣"""
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [2]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana',4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]
Order(joe, cart, FidelityPromo())
Order(ann, cart, FidelityPromo())
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, BulkItemPromo())
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.000000 due: 9.300000>

In [3]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity
    
class Order:
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
            return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:2f} due: {:2f}>'
        return fmt.format(self.total(), self.due())

def fidelity_promo(order): 
    """为积分为1000或以上的顾客提供5%折扣"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """单个商品为20个或以上时提供10%折扣"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

def large_order_promo(order):
    """订单中的不同商品达到10个或以上时提供7%折扣"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [4]:
Order(joe, cart, large_order_promo)

<Order total: 42.000000 due: 42.000000>

In [7]:
#promos = [fidelity_promo, bulk_item_promo large_order_promo]
promos = [globals()[name] for name in globals() 
                if name.endswith('_promo')
                 and name != 'best_promo']

def best_promo(order):
    """选择可用的最佳折扣"""
    return max(promo(order) for promo in promos)

In [6]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'from abc import ABC, abstractmethod\nfrom collections import namedtuple\n\nCustomer= namedtuple(\'Customer\', \'name fidelity\')\n\nclass LineItem:\n    \n    def __init__(self, product, quantity, price):\n        self.product = product\n        self.quantity = quantity\n        self.price = price\n    \n    def total(self):\n        return self.price * self.quantity\n    \nclass Order:\n    \n    def __init__(self, customer, cart, promotion=None):\n        self.customer = customer\n        self.cart = list(cart)\n        self.promotion = promotion\n        \n    def total(self):\n        if not hasattr(self, \'__total\'):\n            self.__total = sum(item.total() for item in self.cart)\n            return s